In [1]:
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    Output,
    ClassificationMetrics,
    Metrics,
    HTML,
    Markdown
)


In [6]:
PROJECT_ID = "kubeflow-demos"  # @param {type:"string"}
PROJECT_NUMBER = "141610882258"
REGION = "us-central1"  # @param {type: "string"}
BUCKET_NAME = "test-fast"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [2]:

@component
def html_visualization(html_artifact: Output[HTML]):
    public_url = 'https://user-images.githubusercontent.com/37026441/140434086-d9e1099b-82c7-4df8-ae25-83fda2929088.png'
    html_content = \
      '<html><head></head><body><h1>Global Feature Importance</h1>\n<img src="{}" width="97%"/></body></html>'.format(public_url)
    with open(html_artifact.path, 'w') as f:
        f.write(html_content)


In [3]:
@component
def markdown_visualization(markdown_artifact: Output[Markdown]):
    import urllib.request

    with urllib.request.urlopen('https://gist.githubusercontent.com/zijianjoy/a288d582e477f8021a1fcffcfd9a1803/raw/68519f72abb59152d92cf891b4719cd95c40e4b6/table_visualization.md') as table:
        markdown_content = table.read().decode('utf-8')
        with open(markdown_artifact.path, 'w') as f:
            f.write(markdown_content)

In [5]:

from kfp.v2 import dsl, compiler
@dsl.pipeline(name=f'metrics-visualization-pipeline-1008')
def metrics_visualization_pipeline():
    html_visualization_op = html_visualization()
    markdown_visualization_op = markdown_visualization()

compiler.Compiler().compile(pipeline_func=metrics_visualization_pipeline, package_path='metrics_visualization_pipeline.json')

/Users/yarkoni/projects/workshop/venv/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1278: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [9]:
import google.cloud.aiplatform as aiplatform

aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project=PROJECT_ID,

    # the Vertex AI region you will use
    # defaults to us-central1
    location=REGION,

    # Google Cloud Storage bucket in same region as location
    # used to stage artifacts
    staging_bucket=BUCKET_URI,

    # the name of the experiment to use to track
    # logged metrics and parameters
    experiment='html-mark',

    # description of the experiment above
    experiment_description='my experiment description'
)

INFO:root:Resource html-mark not found.
INFO:root:Creating Resource html-mark


In [10]:
# Instantiate PipelineJob object
pl = aiplatform.PipelineJob(
    display_name="Mark-Html",

    # Whether or not to enable caching
    # True = always cache pipeline step result
    # False = never cache pipeline step result
    # None = defer to cache option for each pipeline component in the pipeline definition
    enable_caching=True,

    # Local or GCS path to a compiled pipeline definition
    template_path='metrics_visualization_pipeline.json',

    # Dictionary containing input parameters for your pipeline
    parameter_values={},

    # GCS path to act as the pipeline root
    pipeline_root=BUCKET_URI,
)

# Execute pipeline in Vertex AI and monitor until completion
pl.run(
  # Email address of service account to use for the pipeline run
  # You must have iam.serviceAccounts.actAs permission on the service account to use it
  #service_account=service_account,

  # Whether this function call should be synchronous (wait for pipeline run to finish before terminating)
  # or asynchronous (return immediately)
  sync=False
)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/141610882258/locations/us-central1/pipelineJobs/metrics-visualization-pipeline-1008-20220405162340
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/141610882258/locations/us-central1/pipelineJobs/metrics-visualization-pipeline-1008-20220405162340')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/metrics-visualization-pipeline-1008-20220405162340?project=141610882258
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/141610882258/locations/us-central1/pipelineJobs/metrics-visualization-pipeline-1008-20220405162340 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline